In [1]:
from composable import pipeable
from composable.strict import map,zipOnto
from toolz.curried import get
import re


In [8]:
with open('Sell_Short_Trades.txt') as f:
    content = f.read()
    f.close()
content

'                                                                                                                \n DOTC                                                                                 RUN DATE : 06/14/17 21:56 \n                                                                                             RPT DATE :06/14/17 \n                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    \n Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n                                                                                                                \n TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n                                                                    /17  /17  256945   

In [9]:
remove_spaces = content.replace(" ","")
remove_spaces

'\nDOTCRUNDATE:06/14/1721:56\nRPTDATE:06/14/17\nTTS0126:SELLSHORTTRADES&SHORTEXEMPT\nSymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr\n\nTradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr\nCERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS\n/17/1725694500\n0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO\nCERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786608704\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786609604\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786978023\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX\n/17/1789288829\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3600002.35

In [10]:
culled_split=remove_spaces.split("\n")
culled_split

['',
 'DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 '',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnit

In [11]:
empty_spots = lambda l: [len(line.strip())==0 for line in l]


In [12]:
empty_spots(culled_split[:10])

[True, False, False, False, False, True, False, False, False, False]

In [13]:
culled_lines = [line for line in culled_split if empty_spots(line)]
culled_lines

['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.0100

In [14]:
example_block = culled_lines
example_block[:10]

['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704']

In [15]:
pagebreak_example = culled_lines[45:58]
pagebreak_example

['854300',
 '1of77',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW1002.3600002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX',
 '/17/1790916802',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX',
 '/17/1790917702',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3650002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX']

In [16]:
[line for line in culled_lines if line.strip().endswith('of77')][:20]

['1of77',
 '2of77',
 '3of77',
 '4of77',
 '5of77',
 '6of77',
 '7of77',
 '8of77',
 '9of77',
 '10of77',
 '11of77',
 '12of77',
 '13of77',
 '14of77',
 '15of77',
 '16of77',
 '17of77',
 '18of77',
 '19of77',
 '20of77']

In [17]:
remove_pagebreaks = [line for line in culled_lines if not line.strip().endswith('of77')]
remove_pagebreaks

['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.0100

In [18]:
remove_commas = [line.replace(",","") for line in remove_pagebreaks]
remove_commas



['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.01000

In [19]:
remove_backslash = [line.replace("/"," ") for line in remove_commas]
remove_backslash

['DOTCRUNDATE:06 14 1721:56',
 'RPTDATE:06 14 17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.01000

In [20]:
data_lines = remove_backslash[3:]
data_lines

['SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 191

In [21]:
working_data = data_lines

In [22]:
first_columnrow = lambda l: [line.strip().startswith('Symbol') for line in l]

In [23]:
first_columnrow(working_data)

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [24]:
first_columnrow_remove = [line for line in working_data if not line.strip().startswith('Symbol')]
first_columnrow_remove


['TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUn

In [25]:
second_columnrow = lambda l: [line.strip().startswith('TradeComm') for line in l]
second_columnrow(first_columnrow_remove)

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [26]:
second_columnrow_remove = [line for line in first_columnrow_remove if not line.strip().startswith('TradeComm')]
second_columnrow_remove

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000

In [27]:
data = second_columnrow_remove
data

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000

In [28]:
numbers_newline = re.compile(r' 17 17.*')

In [29]:
stop_cond = lambda w: len(w)==2

In [30]:
get_layer = lambda d: d[:4] if len(d[3])==6 else d[:3]

In [31]:
get_remaining = lambda r: r[len(get_layer(r)):]

In [115]:
remaining_lines = data

In [109]:
get_layer(remaining_lines)

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO']

In [110]:
remaining_lines = get_remaining(remaining_lines)
remaining_lines

['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790912801',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 

In [36]:
get_layer(remaining_lines)

['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300']

In [ ]:
remaining_lines = data

In [116]:
new_seq = []
while not stop_cond(remaining_lines):
    new_seq = new_seq + [get_layer(remaining_lines)]
    remaining_lines = get_remaining(remaining_lines)


In [117]:
l = new_seq
l

[['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
  ' 17 1725694500',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO'],
 ['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786608704',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786609604',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786978023',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
  ' 17 1789288829',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
  ' 17 1790911901',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=C

In [118]:
joined_data = [",".join(line) for line in l]
joined_data

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 

In [119]:
text_content = "\n".join(joined_data)
text_content

'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO\nCERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\nCERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790912801,0704P

In [120]:
with open('Project_pt2.txt',"w") as outfile:
    outfile.write(text_content)
    

In [121]:
with open('Project_pt2.txt') as f:
    data_parse = f.read()
    f.close()
data_parse

'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO\nCERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\nCERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790912801,0704P

In [122]:
data_parse = data_parse.split('\n')
data_parse

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 

In [123]:
get_symbol = re.compile(r'CERS|S')
trade_symbols = [get_symbol.search(line).group() for line in data_parse]
trade_symbols

['CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 

In [124]:
len(data_parse)

913

In [125]:
len(trade_symbols)

913

In [126]:
get_side = re.compile(r'SSNEW')
trade_side = [get_side.search(line).group() for line in data_parse]
trade_side

['SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',
 'SSNEW',


In [127]:
trade_side_clean = [line.replace('NEW',"") for line in trade_side]
trade_side_clean

['SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',

In [128]:
len(trade_side_clean)

913

In [129]:
get_cxl = re.compile(r'NEW')
trade_cxl = [get_cxl.search(line).group() for line in data_parse]
trade_cxl

['NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',


In [130]:
len(trade_cxl)

913

In [131]:
get_quantity = re.compile(r'2756|1100|1000|2700|99|2800|1487|2013|34|66|90|40|48|46|\d{1,3}')
trade_quantity = [get_quantity.search(line).group() for line in data_parse]
trade_quantity

['2756',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '200',
 '100',
 '300',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '200',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '300',
 '100',
 '100',
 '100',
 '200',
 '100',
 '200',
 '100',
 '12',
 '199',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '200',
 '212',
 '792',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '202',
 '982',
 '742',
 '262',
 '100',
 '100',
 '100',
 '100',
 '100',
 '700',
 '100',
 '100',
 '100',
 '100',
 '100',
 '40',
 '100',
 '100',
 '300',
 '200',
 '100',
 '300',
 '100',
 '200',
 '100',
 '100',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '300',
 '100',
 '100',
 '200',
 '100',
 '100',
 '100',
 '200',
 

In [132]:
len(trade_quantity)

913

In [133]:
get_price = re.compile(r'\d\.\d{1,3}')
trade_price = [get_price.search(line).group() for line in data_parse]
trade_price

['2.400',
 '2.360',
 '2.350',
 '2.350',
 '2.350',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.365',
 '2.365',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',


In [134]:
get_bid= re.compile(r'0\d\.\d\d')
trade_bid = [get_bid.search(line).group() for line in data_parse]
trade_bid

['02.34',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',


In [135]:
trade_bid_fixing= [line.replace('02','2') for line in trade_bid]
trade_bid_fixing

['2.34',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 

In [136]:
trade_bid_cleaned = [line.replace('08',"8") for line in trade_bid_fixing]
trade_bid_cleaned

['2.34',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 

GET ASKING PRICE STILL


In [137]:
get_ask = re.compile(r'\d\.\d{1,7}6\s')
trade_ask = [get_ask.search(line).group() for line in data_parse]
trade_ask

['2.45000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.36000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.37000006 ',
 '2.3700

In [138]:
trade_ask_clean = [line.replace("6 ","") for line in trade_ask]
trade_ask_clean

['2.4500000',
 '2.3700000',
 '2.3700000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3700000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.3600000',
 '2.36

In [139]:
get_TdatsTrade = re.compile(r'1406')
trade_tDats = [get_TdatsTrade.search(line).group() for line in data_parse]
trade_tDats

['1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 

In [140]:
get_Tdats = re.compile(r'1406')
trade_tDats = [get_Tdats.search(line).group() for line in data_parse]
trade_tDats

['1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 

In [141]:
len(trade_tDats)

913

In [233]:
get_datsID = re.compile(r'191\d{1,4}')
trade_datsID = [get_datsID.search(line).group() for line in data_parse]
trade_datsID

['1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '1917061',
 '19

In [236]:
get_tradeID = re.compile(r'499\d{1,5}')
trade_tradeID = [get_tradeID.search(line).group() for line in data_parse]
trade_tradeID

['49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900003',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',
 '49900004',

In [143]:
get_tradetime = re.compile(r'\d{1,2}:\d{1,2}')
trade_tradetime= [get_tradetime.search(line).group() for line in data_parse]
trade_tradetime

['09:30',
 '10:20',
 '10:20',
 '10:20',
 '10:22',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:24',
 '10:25',
 '10:27',
 '10:27',
 '10:27',
 '10:27',
 '10:27',
 '10:29',
 '10:30',
 '10:33',
 '10:33',
 '10:33',
 '10:33',
 '10:36',
 '10:37',
 '10:41',
 '10:43',
 '10:45',
 '10:45',
 '10:45',
 '10:46',
 '10:47',
 '10:47',
 '10:49',
 '10:50',
 '10:51',
 '10:53',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:55',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '10:56',
 '11:15',
 '11:18',
 '11:19',
 '11:19',
 '11:19',
 '11:19',
 '11:19',
 '11:19',
 '11:19',
 '11:19',
 '11:19',


In [144]:
get_tradeSS = re.compile(r'CustSS|ContraSSFREX|ContraSSDTTX|ContraSSMLCO')
trade_tradeSS= [get_tradeSS.search(line).group() for line in data_parse]
trade_tradeSS

['CustSS',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'C

In [145]:
get_tradeComm = re.compile(r'0704')
trade_CommType= [get_tradeComm.search(line).group() for line in data_parse]
trade_CommType

['0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 

In [146]:
len(trade_CommType)

913

In [147]:
get_sourceComm = re.compile(r'PerUnit')
trade_sourceComm= [get_sourceComm.search(line).group() for line in data_parse]
trade_sourceComm

['PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'Pe

In [148]:
len(trade_sourceComm)

913

In [149]:
get_account = re.compile(r'TERM')
trade_account= [get_account.search(line).group() for line in data_parse]
trade_account

['TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 

In [150]:
len(trade_account)

913

In [151]:
get_orderID = re.compile(r'TERM\d.\d{1,5}')
trade_orderID= [get_orderID.search(line).group() for line in data_parse]
trade_orderID

['TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.

In [152]:
trade_orderID_clean = [line.replace('TERM',"") for line in trade_orderID]
trade_orderID_clean

['0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.

In [153]:
get_gtl = re.compile(r'AABBC|EEFFGG')
trade_gtl= [get_gtl.search(line).group() for line in data_parse]
trade_gtl

['AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 '

In [154]:
len(trade_gtl)

913

In [155]:
get_trailerInfo= re.compile(r'17\d{1,15}#')
trade_trailerInfo= [get_trailerInfo.search(line).group() for line in data_parse]
trade_trailerInfo

['1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900003#',
 '1706149900

In [156]:
trade_trailerInfo_clean = [line.replace('#',"") for line in trade_trailerInfo]
trade_trailerInfo_clean

['1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '17061499

In [157]:
get_trailersymbol= re.compile(r'STA=.+[A-Z]')
trade_trailersymbol= [get_trailersymbol.search(line).group() for line in data_parse]
trade_trailersymbol

['STA=GSCO',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',
 'STA=CERS',

In [158]:
trade_trailerSymbol_clean = [line.replace("STA=","") for line in trade_trailersymbol]
trade_trailerSymbol_clean

['GSCO',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 'MSCO',
 

In [159]:
get_clr= re.compile(r'17\s17\d{1,8}')
trade_clr= [get_clr.search(line).group() for line in data_parse]
trade_clr

['17 1725694500',
 '17 1786608704',
 '17 1786609604',
 '17 1786978023',
 '17 1789288829',
 '17 1790911901',
 '17 1790912801',
 '17 1790914001',
 '17 1790914301',
 '17 1790914801',
 '17 1790915402',
 '17 1790916802',
 '17 1790917702',
 '17 1790918802',
 '17 1790919002',
 '17 1790919302',
 '17 1790919602',
 '17 1790921902',
 '17 1790922402',
 '17 1790967204',
 '17 1791089211',
 '17 1793054059',
 '17 1793998202',
 '17 1793999302',
 '17 1793999502',
 '17 1793999802',
 '17 1794432131',
 '17 1796087630',
 '17 1796901200',
 '17 1701099807',
 '17 1701100807',
 '17 1701101007',
 '17 1701669732',
 '17 1704958307',
 '17 1706432735',
 '17 1710976137',
 '17 1712910807',
 '17 1715015300',
 '17 1715016100',
 '17 1715752737',
 '17 1716677920',
 '17 1717914109',
 '17 1717918809',
 '17 1721052754',
 '17 1721892145',
 '17 1722191602',
 '17 1724466350',
 '17 1725929338',
 '17 1725930538',
 '17 1725931038',
 '17 1725931438',
 '17 1725931838',
 '17 1725933438',
 '17 1726152545',
 '17 1726421956',
 '17 17264

In [160]:
trade_clr_clean = [line[-2:] for line in trade_clr]
trade_clr_clean

['00',
 '04',
 '04',
 '23',
 '29',
 '01',
 '01',
 '01',
 '01',
 '01',
 '02',
 '02',
 '02',
 '02',
 '02',
 '02',
 '02',
 '02',
 '02',
 '04',
 '11',
 '59',
 '02',
 '02',
 '02',
 '02',
 '31',
 '30',
 '00',
 '07',
 '07',
 '07',
 '32',
 '07',
 '35',
 '37',
 '07',
 '00',
 '00',
 '37',
 '20',
 '09',
 '09',
 '54',
 '45',
 '02',
 '50',
 '38',
 '38',
 '38',
 '38',
 '38',
 '38',
 '45',
 '56',
 '56',
 '56',
 '56',
 '56',
 '56',
 '56',
 '56',
 '56',
 '57',
 '57',
 '57',
 '57',
 '59',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '35',
 '36',
 '36',
 '36',
 '36',
 '36',
 '36',
 '36',
 '20',
 '19',
 '23',
 '23',
 '23',
 '23',
 '25',
 '28',
 '28',
 '28',
 '28',
 '28',
 '28',
 '28',
 '28',
 '29',
 '29',
 '29',
 '29',
 '29',
 '29',
 '29',
 '29',
 '29',
 '30',
 '31',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '35',
 '39',
 '39',
 '46',
 '46',
 '46',
 '46',
 '46',
 '46',
 '46',
 '46',
 '47',
 '47',
 '49',
 '51',
 '54',
 '54',
 '54',

In [161]:
get_undertrailer= re.compile(r'17\s17\d{1,6}')
trade_underTrailerNumbers= [get_undertrailer.search(line).group() for line in data_parse]
trade_underTrailerNumbers

['17 17256945',
 '17 17866087',
 '17 17866096',
 '17 17869780',
 '17 17892888',
 '17 17909119',
 '17 17909128',
 '17 17909140',
 '17 17909143',
 '17 17909148',
 '17 17909154',
 '17 17909168',
 '17 17909177',
 '17 17909188',
 '17 17909190',
 '17 17909193',
 '17 17909196',
 '17 17909219',
 '17 17909224',
 '17 17909672',
 '17 17910892',
 '17 17930540',
 '17 17939982',
 '17 17939993',
 '17 17939995',
 '17 17939998',
 '17 17944321',
 '17 17960876',
 '17 17969012',
 '17 17010998',
 '17 17011008',
 '17 17011010',
 '17 17016697',
 '17 17049583',
 '17 17064327',
 '17 17109761',
 '17 17129108',
 '17 17150153',
 '17 17150161',
 '17 17157527',
 '17 17166779',
 '17 17179141',
 '17 17179188',
 '17 17210527',
 '17 17218921',
 '17 17221916',
 '17 17244663',
 '17 17259293',
 '17 17259305',
 '17 17259310',
 '17 17259314',
 '17 17259318',
 '17 17259334',
 '17 17261525',
 '17 17264219',
 '17 17264232',
 '17 17264238',
 '17 17264241',
 '17 17264245',
 '17 17264248',
 '17 17264269',
 '17 17264273',
 '17 172

In [162]:
trade_underTrailers_clean = [line.replace('17 17',"") for line in trade_underTrailerNumbers]
trade_underTrailers_clean

['256945',
 '866087',
 '866096',
 '869780',
 '892888',
 '909119',
 '909128',
 '909140',
 '909143',
 '909148',
 '909154',
 '909168',
 '909177',
 '909188',
 '909190',
 '909193',
 '909196',
 '909219',
 '909224',
 '909672',
 '910892',
 '930540',
 '939982',
 '939993',
 '939995',
 '939998',
 '944321',
 '960876',
 '969012',
 '010998',
 '011008',
 '011010',
 '016697',
 '049583',
 '064327',
 '109761',
 '129108',
 '150153',
 '150161',
 '157527',
 '166779',
 '179141',
 '179188',
 '210527',
 '218921',
 '221916',
 '244663',
 '259293',
 '259305',
 '259310',
 '259314',
 '259318',
 '259334',
 '261525',
 '264219',
 '264232',
 '264238',
 '264241',
 '264245',
 '264248',
 '264269',
 '264273',
 '264275',
 '264364',
 '264376',
 '264378',
 '264382',
 '265085',
 '271744',
 '271752',
 '271764',
 '271770',
 '271775',
 '271779',
 '271792',
 '271805',
 '271811',
 '271815',
 '271819',
 '271824',
 '271843',
 '271845',
 '271857',
 '272010',
 '272021',
 '272023',
 '272027',
 '272189',
 '272197',
 '448173',
 '469512',

In [163]:
get_random= re.compile(r'854300|473640')
trade_random= [get_random.search(line) for line in data_parse]
trade_random

[None,
 <re.Match object; span=(148, 154), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 None,
 <re.Match object; span=(148, 154), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(143, 149), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 None,
 <re.Match object; span=(148, 154), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(143, 149), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(144, 150), match='854300'>,
 <re.Match object; span=(

In [164]:
trade_random_clean = ['0' if get_random.search(line) is None else get_random.search(line).group() for line in data_parse ]
trade_random_clean

['0',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '0',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '473640',
 '473640',
 '473640',
 '473640',


In [166]:
csv_content = tuple(zip(trade_symbols, trade_side_clean, trade_cxl, trade_quantity, trade_price, trade_bid_cleaned, trade_ask_clean, 
                        trade_tDats, trade_datsID, trade_tradetime, trade_tradeSS, trade_CommType, trade_sourceComm,
                        trade_account, trade_orderID_clean, trade_gtl, trade_trailerInfo_clean, trade_trailerSymbol_clean,
                        trade_clr_clean, trade_underTrailers_clean, trade_random_clean))

In [167]:
csv_content

(('CERS',
  'SS',
  'NEW',
  '2756',
  '2.400',
  '2.34',
  '2.4500000',
  '1406',
  '191706149900003',
  '09:30',
  'CustSS',
  '0704',
  'PerUnit',
  'TERM',
  '0.01000',
  'AABBC',
  '1706149900003',
  'GSCO',
  '00',
  '256945',
  '0'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.360',
  '2.36',
  '2.3700000',
  '1406',
  '191706149900003',
  '10:20',
  'ContraSSFREX',
  '0704',
  'PerUnit',
  'TERM',
  '0.01000',
  'AABBC',
  '1706149900003',
  'CERS',
  '04',
  '866087',
  '854300'),
 ('CERS',
  'SS',
  'NEW',
  '200',
  '2.350',
  '2.35',
  '2.3700000',
  '1406',
  '191706149900003',
  '10:20',
  'ContraSSFREX',
  '0704',
  'PerUnit',
  'TERM',
  '0.01000',
  'AABBC',
  '1706149900003',
  'CERS',
  '04',
  '866096',
  '854300'),
 ('CERS',
  'SS',
  'NEW',
  '100',
  '2.350',
  '2.35',
  '2.3600000',
  '1406',
  '191706149900003',
  '10:20',
  'ContraSSFREX',
  '0704',
  'PerUnit',
  'TERM',
  '0.01000',
  'AABBC',
  '1706149900003',
  'CERS',
  '23',
  '869780',
  '854300'),
 ('CER

In [199]:
last_numbers = ['854300' if line[0] == 'CERS' else '473640' for line in csv_content]
last_numbers

['854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',
 '854300',

In [238]:
csv_content_final = tuple(zip(trade_symbols, trade_side_clean, trade_cxl, trade_quantity, trade_price, trade_bid_cleaned, trade_ask_clean, 
                        trade_tDats, trade_datsID, trade_tradeID, trade_tradetime, trade_tradeSS, trade_CommType, trade_sourceComm,
                        trade_account, trade_orderID_clean, trade_gtl, trade_trailerInfo_clean, trade_trailerSymbol_clean,
                        trade_clr_clean, trade_underTrailers_clean, last_numbers))

In [239]:

csv_content_prog = [','.join(line) for line in csv_content_final]
csv_content_prog

['CERS,SS,NEW,2756,2.400,2.34,2.4500000,1406,1917061,49900003,09:30,CustSS,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,GSCO,00,256945,854300',
 'CERS,SS,NEW,100,2.360,2.36,2.3700000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,04,866087,854300',
 'CERS,SS,NEW,200,2.350,2.35,2.3700000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,04,866096,854300',
 'CERS,SS,NEW,100,2.350,2.35,2.3600000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,23,869780,854300',
 'CERS,SS,NEW,100,2.350,2.35,2.3600000,1406,1917061,49900003,10:22,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,29,892888,854300',
 'CERS,SS,NEW,100,2.360,2.36,2.3600000,1406,1917061,49900003,10:24,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,01,909119,854300',
 'CERS,SS,NEW,100,2.360,2.36,2.3600000,1406,1917061,49900003,10:24,ContraSSFREX,0704,PerUnit,TERM,0.01000

In [249]:
csv_content_outfile = '\n'.join(csv_content_prog)
csv_content_outfile

'CERS,SS,NEW,2756,2.400,2.34,2.4500000,1406,1917061,49900003,09:30,CustSS,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,GSCO,00,256945,854300\nCERS,SS,NEW,100,2.360,2.36,2.3700000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,04,866087,854300\nCERS,SS,NEW,200,2.350,2.35,2.3700000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,04,866096,854300\nCERS,SS,NEW,100,2.350,2.35,2.3600000,1406,1917061,49900003,10:20,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,23,869780,854300\nCERS,SS,NEW,100,2.350,2.35,2.3600000,1406,1917061,49900003,10:22,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,29,892888,854300\nCERS,SS,NEW,100,2.360,2.36,2.3600000,1406,1917061,49900003,10:24,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,1706149900003,CERS,01,909119,854300\nCERS,SS,NEW,100,2.360,2.36,2.3600000,1406,1917061,49900003,10:24,ContraSSFREX,0704,PerUnit,TERM,0.01000,AABBC,170614990000

In [241]:
with open('Project_part_3.csv','w') as outfile:
    outfile.write(csv_content_outfile)

In [169]:
import pandas as pd

In [250]:
col_names = ["Symbol", 'Side','Cxl', 'Quantity','Price','Bid','Ask',
             'TDats','Dats','DatsTradeID','TradeTime','TradeSS', 'TradeCommType','SourceCommison',
             'Account','OrderID','GTL','TrailerInfo','TrailerSymbol','Clr','TrailerID','Unkown']

In [251]:
add_headers = pd.read_csv("Project_part_3.csv", names = col_names)
                       
                         

In [252]:
add_headers

,Symbol,Side,Cxl,Quantity,Price,Bid,Ask,TDats,Dats,DatsTradeID,...,TradeCommType,SourceCommison,Account,OrderID,GTL,TrailerInfo,TrailerSymbol,Clr,TrailerID,Unkown
0,CERS,SS,NEW,2756,2.40,2.34,2.45,1406,1917061,49900003,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,GSCO,0,256945,854300
1,CERS,SS,NEW,100,2.36,2.36,2.37,1406,1917061,49900003,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,4,866087,854300
2,CERS,SS,NEW,200,2.35,2.35,2.37,1406,1917061,49900003,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,4,866096,854300
3,CERS,SS,NEW,100,2.35,2.35,2.36,1406,1917061,49900003,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,23,869780,854300
4,CERS,SS,NEW,100,2.35,2.35,2.36,1406,1917061,49900003,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,29,892888,854300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,CERS,SS,NEW,224,2.35,2.34,2.35,1406,1917061,49900006,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,57,843282,854300
909,CERS,SS,NEW,200,2.35,2.35,2.35,1406,1917061,49900006,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,57,843318,854300
910,CERS,SS,NEW,100,2.35,2.35,2.35,1406,1917061,49900006,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,58,843340,854300
911,CERS,SS,NEW,200,2.35,2.35,2.35,1406,1917061,49900006,...,704,PerUnit,TERM,0.01,AABBC,1706149900003,CERS,58,843708,854300


In [253]:
add_headers.to_csv('Project_3_Final.csv')